In [4]:
import sys
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import random
import torch

In [5]:
# Local
cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))
sys.path.append(str(root))

from utils.watertopo import WaterTopo
from utils.simulation import Simulation
from utils.utils import count_parameters, mse_per_timestep

# from models.() import modelClass

In [6]:
#initialize GPU -  In case of windows use cuda instead of nps
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Is CUDA enabled?",torch.cuda.is_available())
print("Number of GPUs",torch.cuda.device_count())

Is CUDA enabled? True
Number of GPUs 1


In [ ]:
# Initialize the model

# best_model = ModelClass(...).to(device)

In [ ]:
# Load the parameters from training
load_path = "./trained_models/unet/..."
# best_model.load_state_dict(torch.load(load_path))


# best_model.eval()

In [10]:
# Let's try the model recursively!
grid_size = 64
channels = 2
sim_length = 97

# Load in a random simulation
sim = WaterTopo.load_simulations(str(root)+"/data/normalized_data/test1/", 1, grid_size, use_augmented_data=True)[0]

# Select the time step where you want to start
id = 0

# Get de inputs and targets
inputs = np.zeros((1, channels, grid_size, grid_size))
inputs[0, 0, :, :] = sim.topography
inputs[0, 1, :, :] = sim.return_timestep(id)
inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
inputs.cpu()

targets = sim.wd
targets = torch.tensor(targets, dtype=torch.float32).to(device)
targets[0,:,:] = inputs[0,1,:,:]

# initialize the outputs
outputs = torch.zeros(targets.shape).to(device)
outputs[0,:,:] = inputs[0,1,:,:]

# run the model
for t in range(1, sim_length):
   outputs[t,:,:] = best_model(inputs)
   inputs[0,1,:,:] = outputs[t,:,:]

mse = mse_per_timestep(targets, outputs)

NameError: name 'best_model' is not defined

In [11]:
# Let's plot the MSE
fig, ax = plt.subplots()
ax.plot(np.arange(0, len(mse)), mse, label="MSE")
ax.set_xlabel("time steps")
ax.set_ylabel("MSE")
ax.set_title("... - MSE per timestep")

: 

In [ ]:
# Anim
ani1 = WaterTopo(topo, outputs.cpu().detach().numpy()).plot_animation()
ani2 = WaterTopo(topo, targets.cpu().detach().numpy()).plot_animation()